In [1]:
import os 
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import math
import SimpleITK as sitk
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

from keras import utils
from keras.preprocessing import image as keras_image
from keras.preprocessing.image import ImageDataGenerator



/Users/qinwenhuang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [205]:
from __future__ import division, print_function

from keras.layers import Input, Conv2D, Conv2DTranspose
from keras.layers import MaxPooling2D, Cropping2D, Concatenate
from keras.layers import Lambda, Activation, BatchNormalization, Dropout
from keras.models import Model
from keras import backend as K


In [101]:
import cv2

In [198]:
class UNet(object):
    def __init__(self, img_row, img_col):
        self.img_row = img_row
        self.img_col = img_col
        
    def unet(self):
        inputs = Input((self.img_row, self.img_col,1))
        #first convolutional layer - output 64 filters, kernel size 3*3, use relu activation function and he normal kernel
        #this draws samples from a truncated normal distribution centered on 0 with stdec = sqrt(2/# of input units in weight tensor)
        #used same padding to make sure output has the same length as input
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
        print ("conv1 shape:",conv1.shape)
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
        print ("conv1 shape:",conv1.shape)
        #then we do a maxpooling 
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        print ("pool1 shape:",pool1.shape)
        
        #then we takes in the pooled output and feed into another convolutional layers with 128 filters
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
        print ("conv2 shape:",conv2.shape)
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
        print ("conv2 shape:",conv2.shape)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        print ("pool2 shape:",pool2.shape)
        
        #then we do another convolutional layer with 256 filters
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
        print ("conv3 shape:",conv3.shape)
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
        print ("conv3 shape:",conv3.shape)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv2)
        print ("pool3 shape:",pool3.shape)
        
        #in order to prevent vanishing gradient problem, we apply dropout in the 4th convolutional layer
        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
        print ("conv4 shape:",conv3.shape)
        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
        print ("conv4 shape:",conv3.shape)
        drop4 = Dropout(0.5)(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv2)
        print ("pool4 shape:",pool3.shape)
        
        #same for 5th convolutional layer
        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
        drop5 = Dropout(0.5)(conv5)
        
        #we then start upconv output of conv 5 (2*2 filter) -> then copy 512 filter conv layer and merge
        up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
        merge6 = merge([drop4,up6], mode = 'concat', concat_axis = 3)
        #then we do convolution again
        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
        
        #we then upconv output of conv 6 (2*2 filter)-> then copy 256 filter conv layer and merge -> conv 2 times
        up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
        merge7 = merge([conv3,up7], mode = 'concat', concat_axis = 3)
        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
        
        #upconv output of conv7 (2*2 filter) -> copy 128 filter conv layer and merge -> conv 2 tiems
        up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
        merge8 = merge([conv2,up8], mode = 'concat', concat_axis = 3)
        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
        
        #upconv output of conv8 (2*2 filter) -> copy 64 filter conv layer and merge -> conv 2 times
        up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
        merge9 = merge([conv1,up9], mode = 'concat', concat_axis = 3)
        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
        
        model = Model(input = inputs, output = conv10)

        model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

        return model

        
        

In [224]:
def downsampling_block(input_tensor, filters, padding='valid',
                       batchnorm=False, dropout=0.0):
    batch, height, width, _ = K.int_shape(input_tensor)
    print(K.shape)
    print(height)
    print(width)
    print(batch)
    #assert height % 2 == 0
    #assert width % 2 == 0

    x = Conv2D(filters, kernel_size=(3,3), padding=padding)(input_tensor)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    x = Conv2D(filters, kernel_size=(3,3), padding=padding)(x)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    return MaxPooling2D(pool_size=(2,2))(x), x

def upsampling_block(input_tensor, skip_tensor, filters, padding='valid',
                     batchnorm=False, dropout=0.0):
    x = Conv2DTranspose(filters, kernel_size=(2,2), strides=(2,2))(input_tensor)

    # compute amount of cropping needed for skip_tensor
    _, x_height, x_width, _ = K.int_shape(x)
    _, s_height, s_width, _ = K.int_shape(skip_tensor)
    h_crop = s_height - x_height
    w_crop = s_width - x_width
    assert h_crop >= 0
    assert w_crop >= 0
    if h_crop == 0 and w_crop == 0:
        y = skip_tensor
    else:
        cropping = ((h_crop//2, h_crop - h_crop//2), (w_crop//2, w_crop - w_crop//2))
        y = Cropping2D(cropping=cropping)(skip_tensor)

    x = Concatenate()([x, y])

    x = Conv2D(filters, kernel_size=(3,3), padding=padding)(x)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    x = Conv2D(filters, kernel_size=(3,3), padding=padding)(x)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    return x

def unet(height, width, channels, classes, features=64, depth=4,
         temperature=1.0, padding='valid', batchnorm=False, dropout=0.0):
    """Generate U-Net model introduced in
      "U-Net: Convolutional Networks for Biomedical Image Segmentation"
      O. Ronneberger, P. Fischer, T. Brox (2015)
    Arbitrary number of input channels and output classes are supported.
    Arguments:
      height  - input image height (pixels)
      width   - input image width  (pixels)
      channels - input image features (1 for grayscale, 3 for RGB)
      classes - number of output classes (2 in paper)
      features - number of output features for first convolution (64 in paper)
          Number of features double after each down sampling block
      depth  - number of downsampling operations (4 in paper)
      padding - 'valid' (used in paper) or 'same'
      batchnorm - include batch normalization layers before activations
      dropout - fraction of units to dropout, 0 to keep all units
    Output:
      U-Net model expecting input shape (height, width, maps) and generates
      output with shape (output_height, output_width, classes). If padding is
      'same', then output_height = height and output_width = width.
    """
    x = Input(shape=(height, width, channels))
    inputs = x

    skips = []
    for i in range(depth):
        x, x0 = downsampling_block(x, features, padding,
                                   batchnorm, dropout)
        skips.append(x0)
        features *= 2

    x = Conv2D(filters=features, kernel_size=(3,3), padding=padding)(x)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    x = Conv2D(filters=features, kernel_size=(3,3), padding=padding)(x)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    for i in reversed(range(depth)):
        features //= 2
        x = upsampling_block(x, skips[i], features, padding,
                             batchnorm, dropout)

    x = Conv2D(filters=classes, kernel_size=(1,1))(x)

    logits = Lambda(lambda z: z/temperature)(x)
    probabilities = Activation('softmax')(logits)

    return Model(inputs=inputs, outputs=probabilities)

In [3]:
def load_nrrd(full_path_filename):

	data = sitk.ReadImage(full_path_filename)
	data = sitk.Cast(sitk.RescaleIntensity(data),sitk.sitkUInt8)
	data = sitk.GetArrayFromImage(data)

	return(data)

In [4]:
#decode mask 	
def run_length_decoding(run_lengths,img):
    
    h,w = img.shape
    mask = np.zeros(h*w)
    if run_lengths == '[\n]':
        pass
    else:
        run_lengths_s = run_lengths[0].split()
        #print(run_lengths_s)
        for i in range(len(run_lengths_s)):
            #even number is index and odd number is # of consecutive tags
            if i%2 == 0:
                #print(i)
                mask[(int(run_lengths_s[i])-1):(int(run_lengths_s[i])+int(run_lengths_s[i+1])-1)] = 1
        mask = mask.reshape((h,w)).T
        

    return mask

In [5]:
train_file = '/Users/qinwenhuang/Documents/AtriaSeg_2018_training/train_labels.csv'

In [6]:
import pandas as pd

In [7]:
train_labels = {}
with open (train_file,'r') as f:
    for ind,line in enumerate(f):
        if ind > 0:
            line = line.split(',')
            train_labels[(line[0])]=line[1:]
            

In [8]:
train_labels

{'06SR5RBREL16DQ6M8LWS_Slice_0': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_1': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_2': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_3': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_4': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_5': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_6': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_7': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_8': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_9': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_10': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_11': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_12': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_13': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_14': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_15': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_16': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_17': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_18': ['\n'],
 '06SR5RBREL16DQ6M8LWS_Slice_19': ['154648 5 155222 9 155797 11 156372 13 156948 14 157523 15 158099 16 158674 18 159250 18 159825 20 160401 20 160976 21 161551 22 162126 23 162701 24 163277 23 163852 24 164427 2

In [9]:
#save dicitonary to pickles so if we want to use it we don't need to run the code again
import pickle
pickle.dump(train_labels, open( "train_labels.p", "wb" ))

In [10]:
#data augmentation
import cv2

In [11]:
def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask


In [12]:
def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask

In [152]:
class dataLoading(object):
    """
    data directory structure
    TrainingSet/
    each patient/
    mask.nrrd, original.nrrd
    """
    def __init__(self, directory, mask_dict):
        self.directory = directory
        self.mask_dict = mask_dict
        self.mris = []
        self.mri_names = []
        self.masks = []
        #self.load_images()
        #self.load_masks()
    def load_images(self):
        """
        load all images from training set 
        go through subdirectories and get lgemri.nrrd, which represents patients original mri images
        uses load_nrrd function 
        retun mri matrices and mri names
        """
        
        for root, dirs, files in os.walk(self.directory, topdown = False):
            for name in files:
                if name == 'lgemri.nrrd':
                    #print('yes')
                    patient_name = root[-20:]+'_Slice_'
                    full_name = os.path.join(root,name)
                    single_patient_image = load_nrrd(full_name)
                    num_of_slices = single_patient_image.shape[0]
                    
                    for i in range(num_of_slices):
                        resze_single = cv2.resize(single_patient_image[i],(640,640))
                        #resze_single = resze_single.reshape(resze_single,(640,640,1))
                        #resze_single = np.expand_dims(resze_single, axis=2)
                        self.mris.append(resze_single)
                        self.mri_names.append(patient_name+str(i))
        return self.mris, self.mri_names 
    def load_masks(self):
        """
        covert all masks in rle to matrix format 
        return matrix format mask list
        """
        #self.masks = []
        for idx,name in enumerate(self.mri_names):
            img = self.mris[idx]
            encode_cav = train_labels[name]
            #print(encode_cav)
            output_mask = run_length_decoding(encode_cav,img)
            resize_mask = cv2.resize(output_mask,(640,640))
            self.masks.append(resize_mask)
        return self.masks
            
        
                
                
                
        
    

In [153]:
all_data = dataLoading(topdir, train_labels)

In [154]:
def load_images(directory, mask):
    all_data = dataLoading(directory, mask)
    image, image_name = all_data.load_images()
    masks = all_data.load_masks()
    return image, masks

In [155]:
image, masks = load_images(topdir, train_labels)

In [56]:
b = image[0].astype('float64')

In [98]:
img = image[:10]

In [135]:
masks = np.dstack(masks)

In [156]:
image = np.dstack(image)

In [157]:
y=np.rollaxis(image,-1)

In [159]:
resze_single = np.expand_dims(y, axis=3)

In [245]:
stacked = np.concatenate((b, b), axis=2)

In [246]:
stacked.shape

(640, 640, 2)

In [162]:
resze_single[0].shape

(640, 640, 1)

In [243]:
b = resze_single[0]

array([[[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [2, 2],
        [1, 1],
        [0, 0]],

       ...,

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]]], dtype=uint8)

In [137]:
x = np.rollaxis(masks,-1)

In [138]:
x.shape

(968, 640, 640)

In [15]:
topdir = '/Users/qinwenhuang/Documents/autoseg/Mini_Training'

Different loss functions (dice, jaccard coefficients)

In [18]:
from __future__ import division, print_function

from keras import backend as K

In [19]:
#to compute dice coefficient
def soft_sorensen_dice(y_true, y_pred, axis=None, smooth=1):
    intersect = K.sum(y_true * y_pred, axis=axis)
    area_true = K.sum(y_true, axis=axis)
    area_pred = K.sum(y_pred, axis=axis)
    return (2 * intersect + smooth) / (area_true + area_pred + smooth)
    
def hard_sorensen_dice(y_true, y_pred, axis=None, smooth=1):
    y_true_int = K.round(y_true)
    y_pred_int = K.round(y_pred)
    return soft_sorensen_dice(y_true_int, y_pred_int, axis, smooth)

In [20]:
def sorensen_dice_loss(y_true, y_pred, weights):
    # Input tensors have shape (batch_size, height, width, classes)
    # must input list of weights with length equal to number of classes
    #
    # Ex: for simple binary classification, with the 0th mask
    # corresponding to the background and the 1st mask corresponding
    # to the object of interest, we set weights = [0, 1]
    batch_dice_coefs = soft_sorensen_dice(y_true, y_pred, axis=[1, 2])
    dice_coefs = K.mean(batch_dice_coefs, axis=0)
    w = K.constant(weights) / sum(weights)
    return 1 - K.sum(w * dice_coefs)

def soft_jaccard(y_true, y_pred, axis=None, smooth=1):
    intersect = K.sum(y_true * y_pred, axis=axis)
    area_true = K.sum(y_true, axis=axis)
    area_pred = K.sum(y_pred, axis=axis)
    union = area_true + area_pred - intersection
    return (intersect + smooth) / (union + smooth)

def hard_jaccard(y_true, y_pred, axis=None, smooth=1):
    y_true_int = K.round(y_true)
    y_pred_int = K.round(y_pred)
    return soft_jaccard(y_true_int, y_pred_int, axis, smooth)

In [21]:
def jaccard_loss(y_true, y_pred, weights):
    batch_jaccard_coefs = soft_jaccard(y_true, y_pred, axis=[1, 2])
    jaccard_coefs = K.mean(batch_jaccard_coefs, axis=0)
    w = K.constant(weights) / sum(weights)
    return 1 - K.sum(w * jaccard_coefs)

def weighted_categorical_crossentropy(y_true, y_pred, weights, epsilon=1e-8):
    ndim = K.ndim(y_pred)
    ncategory = K.int_shape(y_pred)[-1]
    # scale predictions so class probabilities of each pixel sum to 1
    y_pred /= K.sum(y_pred, axis=(ndim-1), keepdims=True)
    y_pred = K.clip(y_pred, epsilon, 1-epsilon)
    w = K.constant(weights) * (ncategory / sum(weights))
    # first, average over all axis except classes
    cross_entropies = -K.mean(y_true * K.log(y_pred), axis=tuple(range(ndim-1)))
    return K.sum(w * cross_entropies)


In [252]:
def elastic_transform(image, alpha, sigma, mode, random_state=None):
    """Elastic deformation of images as described in [Simard2003].
       Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis"
    """
    assert len(image.shape)== 3

    if random_state is None:
        random_state = np.random.RandomState(None)

    height, width, channels = image.shape

    dx = gaussian_filter(2*random_state.rand(height, width) - 1,
                         sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter(2*random_state.rand(height, width) - 1,
                         sigma, mode="constant", cval=0) * alpha

    x, y = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
    indices = (np.repeat(np.ravel(x+dx), channels),
               np.repeat(np.ravel(y+dy), channels),
               np.tile(np.arange(channels), height*width))
    
    values = map_coordinates(image, indices, order=1, mode=mode)

    return values.reshape((height, width, channels))

In [255]:
#create iterator for better data iteration - this is for loading data to Neural Nets  
class Iterator(object):
    def __init__(self, images, masks, batch_size,
                 shuffle=True,
                 rotation_range=90,
                 width_shift_range=0.1,
                 height_shift_range=0.1,
                 shear_range=0.1,
                 zoom_range=0.01,
                 fill_mode='nearest',
                 alpha=500,
                 sigma=20):
        self.images = images
        self.masks = masks
        self.batch_size = batch_size
        self.shuffle = shuffle
        augment_options = {
            'rotation_range': rotation_range,
            'width_shift_range': width_shift_range,
            'height_shift_range': height_shift_range,
            'shear_range': shear_range,
            'zoom_range': zoom_range,
            'fill_mode': fill_mode,
        }
        self.idg = ImageDataGenerator(**augment_options)
        self.alpha = alpha
        self.sigma = sigma
        self.fill_mode = fill_mode
        self.i = 0
        self.index = np.arange(len(images))
        if shuffle:
            np.random.shuffle(self.index)

    def __next__(self):
        return self.next()

    def next(self):
        # compute how many images to output in this batch
        start = self.i
        end = min(start + self.batch_size, len(self.images))
        print('what')
        print(self.batch_size)
        augmented_images = []
        augmented_masks = []
        for n in self.index[start:end]:
            image = self.images[n]
            mask = self.masks[n]
            h,w,channels = image.shape
            #h,w = image.shape

            # stack image + mask together to simultaneously augment
            stacked = np.concatenate((image, mask), axis=2)

            # apply simple affine transforms first using Keras
            augmented = self.idg.random_transform(stacked)

            # maybe apply elastic deformation
            if self.alpha != 0 and self.sigma != 0:
                augmented = elastic_transform(
                    augmented, self.alpha, self.sigma, self.fill_mode)

            # split image and mask back apart
            augmented_image = augmented[:,:,:channels]
            augmented_images.append(augmented_image)
            augmented_mask = np.round(augmented[:,:,channels:])
            augmented_masks.append(augmented_mask)

        self.i += self.batch_size
        if self.i >= len(self.images):
            self.i = 0
            if self.shuffle:
                np.random.shuffle(self.index)

        return np.asarray(augmented_images), np.asarray(augmented_masks)




In [233]:
image, masks = load_images(topdir, train_labels)

In [234]:
for i,img in enumerate(image):
        image[i] = image[i].astype('float64')

In [74]:
img = np.asarray(image)

In [76]:
b = image[0]

In [77]:
normalize(b)

IndexError: too many indices for array

In [236]:
def normalize(x, epsilon=1e-7, axis=1):
    
    x -= np.mean(x, axis=axis, keepdims=True)
    x /= np.std(x, axis=axis, keepdims=True) + epsilon

def create_generators(data_dir, batch_size, validation_split=0.0, mask=train_labels,
                      shuffle_train_val=True, shuffle=True, seed=None,
                      normalize_images=True, augment_training=False,
                      augment_validation=False, augmentation_args={}):
    images, masks = load_images(data_dir, mask)

    # before: type(masks) = uint8 and type(images) = uint8
    # convert images to double-precision
    #images = images.astype('float64')
    for i,img in enumerate(images):
        images[i] = images[i].astype('float64')
    # maybe normalize image
    if normalize_images:
        for i in images:
            normalize(i, axis=1)
    images = np.dstack(images)
    images = np.rollaxis(images,-1)
    masks = np.dstack(masks)
    masks = np.rollaxis(masks, -1)
    images = np.expand_dims(images, axis=3)
    masks = np.expand_dims(masks, axis=3)
    if seed is not None:
        np.random.seed(seed)

    if shuffle_train_val:
        # shuffle images and masks in parallel
        rng_state = np.random.get_state()
        np.random.shuffle(images)
        np.random.set_state(rng_state)
        np.random.shuffle(masks)

    # split out last %(validation_split) of images as validation set
    split_index = int((1-validation_split) * len(images))

    if augment_training:
        train_generator = Iterator(
            images[:split_index], masks[:split_index],
            batch_size, shuffle=shuffle, **augmentation_args)
    else:
        idg = ImageDataGenerator()
        train_generator = idg.flow(images[:split_index], masks[:split_index],
                                   batch_size=batch_size, shuffle=shuffle)

    train_steps_per_epoch = np.ceil(split_index / batch_size)

    if validation_split > 0.0:
        if augment_validation:
            val_generator = Iterator(
                images[split_index:], masks[split_index:],
                batch_size, shuffle=shuffle, **augmentation_args)
        else:
            idg = ImageDataGenerator()
            val_generator = idg.flow(images[split_index:], masks[split_index:],
                                     batch_size=batch_size, shuffle=shuffle)
    else:
        val_generator = None

    val_steps_per_epoch = np.ceil((len(images) - split_index) / batch_size)

    return (train_generator, train_steps_per_epoch,
            val_generator, val_steps_per_epoch)

In [220]:
#you can choose to use different optimizers
#this function takes in optimizer name and arguments for optimizer (e.g. learning rate, decay, momentum)
def use_optimizer(name, args):
    """
    7 possible optimizers - usually adam works the best
    see paper - "Adam, A Method for Stochastic Optimization" by Kingma and Ba, arXiv 1412.6980
    """
    optimizers = {
        'sgd': SGD,
        'rmsprop': RMSprop,
        'adagrad': Adagrad,
        'adadelta': Adadelta,
        'adam': Adam,
        'adamax': Adamax,
        'nadam': Nadam,
    }
    if name not in optimizers:
        raise Exception("Unknown optimizer ({}).".format(name))

    return optimizers[name](**args)

In [221]:
#use train function to train the neural nets - if you want to train the net, just run this function with all arguments
def train(rotation_range, width_shift_rage, height_shift_range, shear_range, zoom_range, fill_mode, datadir, batch_size, validation_split, learning_rate, momentum, decay,optimizer, loss, loss_weights):

    augmentation_args = {
        'rotation_range': rotation_range,
        'width_shift_range': width_shift_range,
        'height_shift_range': height_shift_range,
        'shear_range': shear_range,
        'zoom_range': zoom_range,
        'fill_mode' : fill_mode,

    }
    train_generator, train_steps_per_epoch, val_generator, val_steps_per_epoch = create_generators(
            datadir, batch_size,
            validation_split=validation_split,
            shuffle_train_val=shuffle_train_val,
            shuffle=shuffle,
            seed=seed,
            normalize_images=normalize,
            augment_training=augment_training,
            augment_validation=augment_validation,
            augmentation_args=augmentation_args)
    # get image dimensions from first batch
    images, masks = next(train_generator)
    _,height,width,channels = images.shape
    _,_,_,classes = masks.shape
    #start building model
    model = unet(height=height, width=width, channels=channels, classes=classes,dropout = 0.5)
    model.summary()
    
    optimizer_args = {
        'lr':       learning_rate,
        'decay':    decay
    }
    for k in list(optimizer_args):
        if optimizer_args[k] is None:
            del optimizer_args[k]
    optimizer = use_optimizer(optimizer, optimizer_args)
    
    if loss == 'pixel':
        def lossfunc(y_true, y_pred):
            return weighted_categorical_crossentropy(
                y_true, y_pred, loss_weights)
    elif loss == 'dice':
        def lossfunc(y_true, y_pred):
            return sorensen_dice_loss(y_true, y_pred, loss_weights)
    elif loss == 'jaccard':
        def lossfunc(y_true, y_pred):
            return jaccard_loss(y_true, y_pred, loss_weights)
    else:
        raise Exception("Unknown loss ({})".format(args.loss))


In [237]:
#rotation_range, width_shift_rage, height_shift_range, shear_range, zoom_range, fill_mode, datadir, batch_size, validation_split, learning_rate, momentum, decay,optimizer, loss, loss_weights
rotation_range = 90
width_shift_rage = 0.1
height_shift_range=0.1
shear_range = 0.1
zoom_range = 0.01
fill_mode='nearest'
datadir = topdir
batch_size = 32
validation_split = 0.2
learning_rate = 0.01
#momentum = 0.01
decay = 0.001
optimizer = 'adam'
loss = 'pixel'
loss_weights = 0.1
shuffle_train_val = True
shuffle = True
seed = None
augment_training=True
augment_validation=True
augment_options = {
            'rotation_range': rotation_range,
            'width_shift_range': width_shift_range,
            'height_shift_range': height_shift_range,
            'shear_range': shear_range,
            'zoom_range': zoom_range,
            'fill_mode': fill_mode,
        }
augmentation_args = augment_options

In [256]:
train(rotation_range, width_shift_rage, height_shift_range, shear_range, zoom_range, fill_mode, datadir, batch_size, validation_split, learning_rate, momentum, decay,optimizer, loss, loss_weights)

what
32
<function shape at 0x1167b3ea0>
640
640
None
<function shape at 0x1167b3ea0>
318
318
None
<function shape at 0x1167b3ea0>
157
157
None
<function shape at 0x1167b3ea0>
76
76
None
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 640, 640, 1)  0                                            
__________________________________________________________________________________________________
conv2d_78 (Conv2D)              (None, 638, 638, 64) 640         input_9[0][0]                    
__________________________________________________________________________________________________
activation_66 (Activation)      (None, 638, 638, 64) 0           conv2d_78[0][0]                  
__________________________________________________________________________________________________
dropout_65 (Dropout)  

In [172]:
#compute statistics in each round 
def compute_statistics(model, generator, steps_per_epoch, return_images=False):
    dices = []
    jaccards = []
    predictions = []
    for i in range(steps_per_epoch):
        images, masks_true = next(generator)
        # Normally: masks_pred = model.predict(images)
        # But dilated densenet cannot handle large batch size
        masks_pred = np.concatenate([model.predict(image[None,:,:,:]) for image in images])
        for mask_true, mask_pred in zip(masks_true, masks_pred):
            y_true = mask_true[:,:,1].astype('uint8')
            y_pred = np.round(mask_pred[:,:,1]).astype('uint8')
            dices.append(sorensen_dice(y_true, y_pred))
            jaccards.append(jaccard(y_true, y_pred))
        if return_images:
            for image, mask_true, mask_pred in zip(images, masks_true, masks_pred):
                predictions.append((image[:,:,0], mask_true[:,:,1], mask_pred[:,:,1]))
    print("Dice:    {:.3f} ({:.3f})".format(np.mean(dices), np.std(dices)))
    print("Jaccard: {:.3f} ({:.3f})".format(np.mean(jaccards), np.std(jaccards)))
    return dices, jaccards, predictions

4.0